In [ ]:

import warnings
import os
from src.green_pont import load_json_with_comments
from src.urban_wind import wind_speed_grib, read_cfd_wind, scale_cfd_wind, save_local_wind, plot_maps
from src.plots_jupyter import plot_urban_wind
import ipywidgets as widgets

warnings.filterwarnings("always")

# 1. Manually specify the expected PROJ data path for your Conda environment.
#    REPLACE THIS WITH THE YOUR ACTUAL PATH.
"""

conda_proj_data_path = r"/opt/conda/envs/urbanair/share/proj"

# 2. Delete PROJ_LIB from os.environ first, in case it was already set incorrectly.
if 'PROJ_LIB' in os.environ:
    del os.environ['PROJ_LIB']

# 3. Explicitly set PROJ_LIB to the correct Conda path.
os.environ['PROJ_LIB'] = conda_proj_data_path
print(f"PROJ_LIB manually set to: {os.environ['PROJ_LIB']}")
"""



In [ ]:
cf=load_json_with_comments('etc/settings_wind_maps.json')

path_cfd=cf['path_cfd'] #'/projects/urbanair/DATA/DATA_AntwerpWindStudy/wind_ratios'
angles=cf["angles"]
height=cf["height"]
path = cf['path_meteo'] #'data/meteo_input/'
grib_file= cf['grib_file'] # 'GRIBPFDEOD+$hour$h00m00s'
output_path=cf["output_path"]

masks=[cf['geometry']["buildings"], cf['geometry']["waterbodies"]]  

# read meteo file from meso-scale model
wind_meteo = wind_speed_grib(30, path, grib_file)

# read pre-computed normalized CFD wind ratios
cfd_ratio=read_cfd_wind(path_cfd,angles, height)

# scale meso-scale wind to local urban scale using CFD ratios
wind_local=scale_cfd_wind(wind_meteo, cfd_ratio)

os.makedirs(output_path, exist_ok=True)



saved_files=save_local_wind(wind_local, cfd_ratio, output_path, reproject=True, mask_frames=masks)


#plot_maps(saved_files, path=output_path)


    

In [ ]:
# Define sliders
slider1 = widgets.IntSlider(
    value=1, min=0, max=24, step=1,
    description='Hour of the day:', continuous_update=True
)

# Display all three
display(slider1)




In [ ]:

print(str.zfill(str(slider1.value), 2))

hour=str.zfill(str(slider1.value), 2)
tif_path="data/maps/urban_wind//wind_175_20240811_"+hour+"00_4326.tif"
print(tif_path)

#tif_path = "/projects/urbanair/urbanair_greenpotential/data/maps/recomendation_method3_4326.tif"
plot_urban_wind(tif_path, vmin=0, vmax=0.1, mask_shp="data/geometry/bld_all_fused.shp")